In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import openmeteo_requests
from openmeteo_sdk import Model, WeatherApiResponse, Variable

import requests_cache
import pandas as pd
from retry_requests import retry

import polars as pl
from datetime import datetime, timedelta, timezone

import logging

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

from pipeline.api_wrapper import Forecast

In [3]:
forecaster = Forecast()
hourly_data, current_data = forecaster.get_data(57.3046898, 25.3627957)
display(hourly_data)
display(current_data)

INFO:pipeline.api_wrapper:Getting data for 57.3046898, 25.3627957


created_at,date,model,latitude,longitude,elevation,temperature,wind_speed,wind_direction,relative_humidity,dew_point,apparent_temperature,precipitation,precipitation_probability,rain,showers,snowfall,snow_depth,weather_code,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high
"datetime[μs, UTC]","datetime[μs, UTC]",str,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2025-01-24 19:44:59.689322 UTC,2025-01-24 00:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,1.657,3.612478,175.23645,86.0,-0.43322,-2.430413,0.0,0.0,0.0,0.0,0.0,0.0,3.0,100.0,100.0,100.0,100.0
2025-01-24 19:44:59.689322 UTC,2025-01-24 01:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,1.857,3.255764,169.380402,87.0,-0.077411,-1.989153,0.0,0.0,0.0,0.0,0.0,0.0,3.0,100.0,100.0,100.0,100.0
2025-01-24 19:44:59.689322 UTC,2025-01-24 02:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,1.957,3.501428,178.363464,88.0,0.17874,-1.9785,0.0,0.0,0.0,0.0,0.0,0.0,3.0,100.0,100.0,100.0,100.0
2025-01-24 19:44:59.689322 UTC,2025-01-24 03:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,2.107,3.238827,171.119415,90.0,0.63769,-1.617992,0.0,15.0,0.0,0.0,0.0,0.0,3.0,100.0,100.0,100.0,100.0
2025-01-24 19:44:59.689322 UTC,2025-01-24 04:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,1.857,3.436568,188.365799,92.0,0.69511,-1.962631,0.2,85.0,0.2,0.0,0.0,0.0,61.0,100.0,91.0,100.0,100.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-01-24 19:44:59.763563 UTC,2025-01-30 19:00:00 UTC,"""dmi_seamless""",57.312023,25.36145,115.0,4.45,2.507987,246.501419,98.0,NaN,1.743028,0.0,23.0,0.0,0.0,0.0,NaN,3.0,100.0,91.0,63.0,15.0
2025-01-24 19:44:59.763563 UTC,2025-01-30 20:00:00 UTC,"""dmi_seamless""",57.312023,25.36145,115.0,4.35,2.416609,245.556061,99.0,NaN,1.700019,0.0,21.0,0.0,0.0,0.0,NaN,3.0,100.0,93.0,62.0,20.0
2025-01-24 19:44:59.763563 UTC,2025-01-30 21:00:00 UTC,"""dmi_seamless""",57.312023,25.36145,115.0,4.3,2.325941,244.536697,99.0,NaN,1.687586,0.0,20.0,0.0,0.0,0.0,NaN,3.0,100.0,95.0,63.0,25.0


created_at,date,model,latitude,longitude,elevation,temperature,wind_speed,wind_direction,relative_humidity,dew_point,apparent_temperature,precipitation,precipitation_probability,rain,showers,snowfall,snow_depth,weather_code,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high
"datetime[μs, UTC]","datetime[μs, UTC]",str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2025-01-24 19:44:59.768681 UTC,2025-01-24 19:30:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,1.05,3.101613,178.152435,91.0,-0.254747,-2.742455,0.0,0.0,0.0,0.0,0.0,0.0,2.0,71.0,63.0,0.0,24.0
2025-01-24 19:44:59.770297 UTC,2025-01-24 19:30:00 UTC,"""gfs_seamless""",57.34462,25.3125,115.0,1.9,4.110961,184.18483,87.0,-0.035088,-2.388623,0.0,0.0,0.0,0.0,0.0,0.0,3.0,100.0,100.0,0.0,100.0
2025-01-24 19:44:59.771520 UTC,2025-01-24 19:30:00 UTC,"""meteofrance_seamless""",57.299999,25.400002,115.0,1.1,4.341659,194.676437,94.0,0.242054,-3.266943,0.0,0.0,0.0,0.0,0.0,NaN,3.0,95.0,53.0,0.0,10.0
2025-01-24 19:44:59.772658 UTC,2025-01-24 19:30:00 UTC,"""ecmwf_ifs025""",57.25,25.25,115.0,1.6,4.472136,190.304794,92.0,NaN,-2.804053,0.0,0.0,0.0,0.0,0.0,NaN,2.0,63.0,21.0,48.0,50.0
2025-01-24 19:44:59.773796 UTC,2025-01-24 19:30:00 UTC,"""ecmwf_aifs025""",57.25,25.25,115.0,1.75,4.554119,188.841736,92.0,NaN,-2.673391,0.0,NaN,0.0,0.0,0.0,NaN,0.0,15.0,14.0,4.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-01-24 19:44:59.778145 UTC,2025-01-24 19:30:00 UTC,"""gem_seamless""",57.300003,25.350006,115.0,0.85,4.7,203.0,92.0,-0.302376,-3.788868,0.0,0.0,0.0,0.0,0.0,0.0,2.0,71.0,63.60165,0.0,0.0
2025-01-24 19:44:59.779138 UTC,2025-01-24 19:30:00 UTC,"""bom_access_global""",57.363281,25.400391,115.0,1.85,3.9,193.0,94.0,0.986784,-2.165499,0.0,0.0,0.0,0.0,0.0,0.0,3.0,98.0,75.0,0.0,91.0
2025-01-24 19:44:59.780147 UTC,2025-01-24 19:30:00 UTC,"""cma_grapes_global""",57.3125,25.375,115.0,-3.55,4.031129,187.124924,95.0,-4.234833,-8.362971,0.0,NaN,0.0,0.0,0.0,0.23,0.0,4.0,0.0,0.0,4.0


In [4]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 57.3046898,
	"longitude": 25.3627957,
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m", "relative_humidity_2m"],
    "wind_speed_unit": "ms",
    # "models": all_models
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")



Coordinates 57.30215835571289°N 25.3603515625°E
Elevation 115.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [5]:
def model_to_name(code):
    """convert Model to name"""
    for name, value in Model.Model.__dict__.items():
        if value == code and not name.startswith("_"):
            return name
    return None

for response in responses:
    print(model_to_name(response.Model()))

valid_models = list(set([model_to_name(response.Model()) for response in responses]))


best_match


In [6]:
response.Hourly().Variables(0).Variable()

47

In [7]:
import csv

with open("valid_models.csv", 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(valid_models)

In [8]:
dir(response.Hourly().Variables(1))
response.Hourly().Variables(1).Variable()

59

In [9]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
# hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

# hourly_data = {"date": pd.date_range(
# 	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
# 	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
# 	freq = pd.Timedelta(seconds = hourly.Interval()),
# 	inclusive = "left"
# )}
# hourly_data["temperature_2m"] = hourly_temperature_2m
# hourly_data["wind_speed_10m"] = hourly.Variables(1).ValuesAsNumpy()
# hourly_data["wind_direction_10m"] = hourly.Variables(2).ValuesAsNumpy()
# hourly_data["relative_humidity_2m"] = hourly.Variables(3).ValuesAsNumpy()

# hourly_dataframe = pd.DataFrame(data = hourly_data)
# print(hourly_dataframe)

start = datetime.fromtimestamp(hourly.Time(), timezone.utc)
end = datetime.fromtimestamp(hourly.TimeEnd(), timezone.utc)
freq = timedelta(seconds = hourly.Interval())

hourly_dataframe_pl = pl.select(
    date = pl.datetime_range(start, end, freq, closed = "left"),
    temperature_2m = hourly.Variables(0).ValuesAsNumpy(),
    wind_speed_10m = hourly.Variables(1).ValuesAsNumpy(),
    wind_direction_10m = hourly.Variables(2).ValuesAsNumpy(),
    relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
)
hourly_dataframe_pl

date,temperature_2m,wind_speed_10m,wind_direction_10m,relative_humidity_2m
"datetime[μs, UTC]",f32,f32,f32,f32
2025-01-24 00:00:00 UTC,1.6525,4.4,175.0,89.0
2025-01-24 01:00:00 UTC,1.7025,4.7,188.0,90.0
2025-01-24 02:00:00 UTC,1.6025,4.2,184.0,91.0
2025-01-24 03:00:00 UTC,1.4025,3.5,177.0,98.0
2025-01-24 04:00:00 UTC,1.2525,3.3,187.0,97.0
…,…,…,…,…
2025-01-30 19:00:00 UTC,4.45,2.507987,246.501419,98.0
2025-01-30 20:00:00 UTC,4.35,2.416609,245.556061,99.0
2025-01-30 21:00:00 UTC,4.3,2.325941,244.536697,99.0


In [10]:
hourly.Time()

1737676800

In [11]:
import dagster
class MDConfig(dagster.Config):
    # hourly_data_path: str = os.path.join(os.path.dirname(__file__), "data", "hourly_data.csv")
    hourly_data_columns: list[str] = ["temperature_2m", "wind_speed_10m", "wind_direction_10m", "relative_humidity_2m"]

config = MDConfig()

params = {
	"latitude": 57.3046898,
	"longitude": 25.3627957,
	"hourly": config.hourly_data_columns,
    # "hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m", "relative_humidity_2m"],
    "wind_speed_unit": "ms",
    "timezone": "Europe/Riga"
}

for key, value in params.items():
    print(f"{key}: {value}")
    print(type(value))

responses = openmeteo.weather_api(url, params=params)


latitude: 57.3046898
<class 'float'>
longitude: 25.3627957
<class 'float'>
hourly: ['temperature_2m', 'wind_speed_10m', 'wind_direction_10m', 'relative_humidity_2m']
<class 'list'>
wind_speed_unit: ms
<class 'str'>
timezone: Europe/Riga
<class 'str'>


In [12]:
extremes = {
    "north": 58.085137,
    "south": 55.674971,
    "east": 28.241043,
    "west": 20.970154
}

In [13]:
(extremes["north"] - extremes["south"]) / (extremes["east"] - extremes["west"])

0.33148161112073143

In [14]:
n_long = 45

In [16]:
import geopandas as gpd

gpd.tools.geocode("Riga, Latvia")

GeocoderUnavailable: HTTPSConnectionPool(host='photon.komoot.io', port=443): Max retries exceeded with url: /api?q=Riga%2C+Latvia&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='photon.komoot.io', port=443): Read timed out. (read timeout=1)"))

In [17]:
import reverse_geocode
reverse_geocode.get((57.3046898, 25.3627957))

{'country_code': 'LV',
 'city': 'Priekuļi',
 'latitude': 57.315,
 'longitude': 25.36147,
 'population': 2532,
 'state': 'Cēsis Municipality',
 'county': 'Priekuļu pagasts',
 'country': 'Latvia'}

In [18]:
import shapely
gpd.tools.reverse_geocode([shapely.geometry.Point(57.3046898, 25.3627957),shapely.geometry.Point(57.3046898, 25.3627957)])

GeocoderUnavailable: HTTPSConnectionPool(host='photon.komoot.io', port=443): Max retries exceeded with url: /reverse?lat=25.3627957&lon=57.3046898&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='photon.komoot.io', port=443): Read timed out. (read timeout=1)"))

In [24]:
gpd.tools.reverse_geocode(  
    [shapely.geometry.Point(-71.0594869, 42.3584697), shapely.geometry.Point(-77.0365305, 38.8977332),
     shapely.geometry.Point(57.315, 25.36147), shapely.geometry.Point(57.3046898, 25.3627957)
    ]
)

,geometry,address
0,POINT (-71.05958 42.35862),"18-32, Tremont Street, 02108, Tremont Street, ..."
1,POINT (-77.03655 38.8977),"White House, 1600, Pennsylvania Avenue Northwe..."
2,POINT EMPTY,None
3,POINT EMPTY,None


In [19]:
gpd.read_file("shapfiles/lv_10km.shp")


,geometry
0,"POLYGON ((4870000 3670000, 4870000 3680000, 48..."
1,"POLYGON ((4870000 3680000, 4870000 3690000, 48..."
2,"POLYGON ((4870000 3690000, 4870000 3700000, 48..."
3,"POLYGON ((4870000 3700000, 4870000 3710000, 48..."
4,"POLYGON ((4870000 3710000, 4870000 3720000, 48..."
...,...
1295,"POLYGON ((5440000 3850000, 5440000 3860000, 54..."
1296,"POLYGON ((5450000 3800000, 5450000 3810000, 54..."
1297,"POLYGON ((5450000 3810000, 5450000 3820000, 54..."
1298,"POLYGON ((5450000 3820000, 5450000 3830000, 54..."


In [20]:
countries = gpd.read_file("shapfiles/world-administrative-boundaries.shp")


In [21]:
countries.head()

,iso3,status,color_code,name,continent,region,iso_3166_1_,french_shor,geometry
0,MNP,US Territory,USA,Northern Mariana Islands,Oceania,Micronesia,MP,Northern Mariana Islands,"MULTIPOLYGON (((145.63331 14.91236, 145.62412 ..."
1,None,Sovereignty unsettled,RUS,Kuril Islands,Asia,Eastern Asia,None,Kuril Islands,"MULTIPOLYGON (((146.68274 43.70777, 146.66664 ..."
2,FRA,Member State,FRA,France,Europe,Western Europe,FR,France,"MULTIPOLYGON (((9.4475 42.68305, 9.45014 42.63..."
3,SRB,Member State,SRB,Serbia,Europe,Southern Europe,RS,Serbie,"POLYGON ((20.26102 46.11485, 20.31403 46.06986..."
4,URY,Member State,URY,Uruguay,Americas,South America,UY,Uruguay,"POLYGON ((-53.3743 -33.74067, -53.39917 -33.75..."


In [22]:
countries[countries["iso3"] == "LVA"]

,iso3,status,color_code,name,continent,region,iso_3166_1_,french_shor,geometry
86,LVA,Member State,LVA,Latvia,Europe,Northern Europe,LV,Lettonie,"POLYGON ((27.37206 57.53564, 27.425 57.53194, ..."


In [23]:
import numpy as np

n_long = 45
n_lat = 15

longitudes = np.linspace(extremes["west"], extremes["east"], n_long)
latitudes = np.linspace(extremes["south"], extremes["north"], n_lat)

longitudes
latitudes

points = [shapely.geometry.Point(long, lat) for long in longitudes for lat in latitudes]

lv_grid = gpd.GeoDataFrame(geometry=points)
lv_grid.crs = "EPSG:4326"

lv_grid


,geometry
0,POINT (20.97015 55.67497)
1,POINT (20.97015 55.84713)
2,POINT (20.97015 56.01928)
3,POINT (20.97015 56.19144)
4,POINT (20.97015 56.36359)
...,...
670,POINT (28.24104 57.39652)
671,POINT (28.24104 57.56867)
672,POINT (28.24104 57.74083)
673,POINT (28.24104 57.91298)


In [24]:
contains = lv_grid.geometry.within(countries[countries["iso3"] == "LVA"].geometry.iloc[0])

lv_grid = lv_grid[contains].reset_index(drop=True)


In [25]:
lv_grid.geometry.iloc[0].coords[0][0]


20.970154

In [26]:
lv_grid["rev_geocode"] = lv_grid.geometry.apply(lambda x: reverse_geocode.get((x.coords[0][1], x.coords[0][0])))

In [27]:
lv_grid["latitude"] = lv_grid.rev_geocode.apply(lambda x: x["latitude"])
lv_grid["longitude"] = lv_grid.rev_geocode.apply(lambda x: x["longitude"])

lv_grid["full_name"] = lv_grid.rev_geocode.apply(lambda x: f"{x.get('state', '')}{', ' if x.get('state') else ''}{x.get('county', '')}{', ' if x.get('county') else ''}{x.get('city', '')}")

lv_grid = lv_grid[lv_grid.rev_geocode.apply(lambda x: x.get("country_code") == "LV")].reset_index(drop=True)


In [28]:
lv_grid.drop_duplicates(subset=["full_name"])

,geometry,rev_geocode,latitude,longitude,full_name
0,POINT (20.97015 56.36359),"{'country_code': 'LV', 'city': 'Nīca', 'latitu...",56.34601,21.06401,"South Kurzeme Municipality, Nīcas pagasts, Nīca"
1,POINT (21.1354 56.19144),"{'country_code': 'LV', 'city': 'Rucava', 'lati...",56.16314,21.16156,"South Kurzeme Municipality, Rucavas pagasts, R..."
3,POINT (21.1354 56.53574),"{'country_code': 'LV', 'city': 'Grobiņa', 'lat...",56.53521,21.16782,"South Kurzeme Municipality, Grobiņa"
6,POINT (21.30065 56.53574),"{'country_code': 'LV', 'city': 'Lieģi', 'latit...",56.58173,21.33399,"South Kurzeme Municipality, Tadaiķu pagasts, L..."
8,POINT (21.30065 56.88005),"{'country_code': 'LV', 'city': 'Pāvilosta', 'l...",56.88791,21.18593,"South Kurzeme Municipality, Pāvilosta"
...,...,...,...,...,...
286,POINT (27.41481 56.01928),"{'country_code': 'LV', 'city': 'Dagda', 'latit...",56.09512,27.53723,"Krāslava Municipality, Dagda"
291,POINT (27.41481 56.88005),"{'country_code': 'LV', 'city': 'Kārsava', 'lat...",56.78405,27.68829,"Ludza Municipality, Kārsava"
297,POINT (27.58005 56.36359),"{'country_code': 'LV', 'city': 'Ludza', 'latit...",56.53958,27.71891,"Ludza Municipality, Ludza"
301,POINT (27.58005 57.05221),"{'country_code': 'LV', 'city': 'Viļaka', 'lati...",57.18458,27.67220,"Balvi Municipality, Viļaka"


In [29]:
lv_grid[lv_grid["full_name"].str.startswith("Cēsis")].full_name.tolist()

['Cēsis Municipality, Stalbes pagasts, Stalbe',
 'Cēsis Municipality, Līgatne',
 'Cēsis Municipality, Līgatne',
 'Cēsis Municipality, Stalbes pagasts, Stalbe',
 'Cēsis Municipality, Drabešu pagasts, Drabeši',
 'Cēsis Municipality, Drabešu pagasts, Drabeši',
 'Cēsis Municipality, Cēsis',
 'Cēsis Municipality, Drabešu pagasts, Drabeši',
 'Cēsis Municipality, Priekuļu pagasts, Priekuļi',
 'Cēsis Municipality, Vecpiebalgas pagasts, Vecpiebalga',
 'Cēsis Municipality, Vecpiebalgas pagasts, Vecpiebalga',
 'Cēsis Municipality, Vecpiebalgas pagasts, Vecpiebalga',
 'Cēsis Municipality, Vecpiebalgas pagasts, Vecpiebalga']

In [43]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

key = '36e53b411cda4663bdf6120faed85a0f'
geocoder = OpenCageGeocode(key)

geocoder.reverse_geocode(lv_grid.latitude.tolist()[0], lv_grid.longitude.tolist()[0])[0].get("components").get("ISO_3166-2")[0]


'LV-112'

In [45]:
geocoder.reverse_geocode(57.315,25.36147)[0].get("components")#.get("ISO_3166-2")

{'ISO_3166-1_alpha-2': 'LV',
 'ISO_3166-1_alpha-3': 'LVA',
 'ISO_3166-2': ['LV-022'],
 '_category': 'building',
 '_normalized_city': 'Priekuļu pagasts',
 '_type': 'building',
 'city': 'Priekuļu pagasts',
 'city_district': 'Priekuļi',
 'continent': 'Europe',
 'country': 'Latvia',
 'country_code': 'lv',
 'house_number': '7',
 'political_union': 'European Union',
 'postcode': 'LV-4126',
 'road': 'Muižas iela',
 'state_district': 'Cēsu novads'}

In [41]:
lv_grid["iso_3166"] = lv_grid.apply(lambda x: geocoder.reverse_geocode(x.latitude, x.longitude)[0].get("components").get("ISO_3166-2")[0], axis=1)

In [42]:
lv_grid.drop_duplicates(subset=["full_name"])[["full_name", "latitude", "longitude", "iso_3166"]].rename(columns={"iso_3166": "iso_3166-2"}).to_csv("lv_grid.csv", index=False)

In [39]:
lv_grid

,geometry,rev_geocode,latitude,longitude,full_name,iso_3166
0,POINT (20.97015 56.36359),"{'country_code': 'LV', 'city': 'Nīca', 'latitu...",56.34601,21.06401,"South Kurzeme Municipality, Nīcas pagasts, Nīca",[LV-112]
1,POINT (21.1354 56.19144),"{'country_code': 'LV', 'city': 'Rucava', 'lati...",56.16314,21.16156,"South Kurzeme Municipality, Rucavas pagasts, R...",[LV-112]
2,POINT (21.1354 56.36359),"{'country_code': 'LV', 'city': 'Nīca', 'latitu...",56.34601,21.06401,"South Kurzeme Municipality, Nīcas pagasts, Nīca",[LV-112]
3,POINT (21.1354 56.53574),"{'country_code': 'LV', 'city': 'Grobiņa', 'lat...",56.53521,21.16782,"South Kurzeme Municipality, Grobiņa",[LV-112]
4,POINT (21.1354 56.7079),"{'country_code': 'LV', 'city': 'Grobiņa', 'lat...",56.53521,21.16782,"South Kurzeme Municipality, Grobiņa",[LV-112]
...,...,...,...,...,...,...
310,POINT (27.7453 57.22436),"{'country_code': 'LV', 'city': 'Viļaka', 'lati...",57.18458,27.67220,"Balvi Municipality, Viļaka",[LV-015]
311,POINT (27.91055 56.36359),"{'country_code': 'LV', 'city': 'Zilupe', 'lati...",56.38616,28.12165,"Ludza Municipality, Zilupe",[LV-058]
312,POINT (27.91055 56.53574),"{'country_code': 'LV', 'city': 'Ludza', 'latit...",56.53958,27.71891,"Ludza Municipality, Ludza",[LV-058]
313,POINT (28.0758 56.36359),"{'country_code': 'LV', 'city': 'Zilupe', 'lati...",56.38616,28.12165,"Ludza Municipality, Zilupe",[LV-058]


In [50]:
import os
os.getcwd()

'/home/linards/Documents/meteo-data/dagster'

In [52]:
# read "superset-lv.geojson"
import json

with open("superset-lv.geojson", "r") as f:
    geojson = json.load(f)

geojson["features"]

[{'type': 'Feature',
  'properties': {'ISO': 'LV-084', 'NAME_1': 'Rujienas'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[25.26350142400011, 58.07513844900008],
     [25.281691528000067, 58.07340728800001],
     [25.29957157400011, 58.065604147],
     [25.30639286300007, 58.05873118100003],
     [25.31827844300011, 58.040618592],
     [25.32458296700011, 58.034753317000096],
     [25.333678019000104, 58.03180776],
     [25.33987919100008, 58.03258290600003],
     [25.346390421000137, 58.034443258000024],
     [25.356519002000084, 58.034753317000096],
     [25.396619914000098, 58.02317779600007],
     [25.41973136714802, 58.008250005422155],
     [25.40683425549645, 57.991310424136714],
     [25.403702900668293, 57.97095661685438],
     [25.37082367317396, 57.9693909394403],
     [25.345772833649335, 57.96625958461209],
     [25.3410758014071, 57.9850477144804],
     [25.326984703781022, 57.991310424136714],
     [25.301933864256398, 57.97095661685438],
     [25.2784487021458

In [70]:
from shapely.geometry import Polygon, MultiPolygon

Polygon(geojson["features"][0]["geometry"]["coordinates"][0]).centroid.coords[0]


(25.271642579859133, 57.92740428432011)

In [78]:
MultiPolygon(geojson["features"][32]["geometry"]["coordinates"]).centroid.coords[0]

(21.039851726294806, 56.5472778049238)

In [83]:
def get_centroid(feature):
    # print(feature["geometry"]["coordinates"][0])
    # print(feature["properties"])
    if feature["geometry"]["type"] == "Polygon":
        centroid = Polygon(feature["geometry"]["coordinates"][0]).centroid.coords[0]
    elif feature["geometry"]["type"] == "MultiPolygon":
        centroid = MultiPolygon(feature["geometry"]["coordinates"]).centroid.coords[0]
    return {
        "iso_3166-2": feature["properties"]["ISO"],
        "name": feature["properties"]["NAME_1"],
        "latitude": centroid[0],
        "longitude": centroid[1]
    }
# centroids = [feature["geometry"] for feature in geojson["features"]]

# centroids

get_centroid(geojson["features"][32])

{'iso_3166-2': 'LV-LPX',
 'name': 'Liepāja',
 'latitude': 21.039851726294806,
 'longitude': 56.5472778049238}

In [87]:
pl.DataFrame([get_centroid(feature) for feature in geojson["features"]]).write_csv("lv_grid.csv")